# Configuration Validation

This notebook validates that all configuration files are correctly formatted and consistent.

## Overview

- Validate regions.yaml
- Validate plotting.yaml
- Check configuration consistency
- Test configuration loading


In [ ]:
import sys
from pathlib import Path
import yaml

# Add project root to path
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))


In [ ]:
# Validate regions.yaml
print("=== Regions Configuration ===")
regions_config_path = project_root / "configs" / "regions.yaml"

try:
    with open(regions_config_path, 'r') as f:
        regions_config = yaml.safe_load(f)

    print(f"✓ Successfully loaded {regions_config_path.name}")
    print(f"  Regions: {len(regions_config.get('regions', {}))}")
    print(f"  Settings: {regions_config.get('settings', {})}")

    # Validate structure
    assert 'regions' in regions_config, "Missing 'regions' key"
    assert 'settings' in regions_config, "Missing 'settings' key"

    print("  ✓ Valid structure")

except Exception as e:
    print(f"✗ Error loading regions.yaml: {e}")


In [ ]:
# Validate plotting.yaml
print("\n=== Plotting Configuration ===")
plotting_config_path = project_root / "configs" / "plotting.yaml"

try:
    with open(plotting_config_path, 'r') as f:
        plotting_config = yaml.safe_load(f)

    print(f"✓ Successfully loaded {plotting_config_path.name}")

    # Check required keys
    if 'no_log_scale_vars' in plotting_config:
        print(f"  no_log_scale_vars: {len(plotting_config['no_log_scale_vars'])} variables")

    if 'region_exclusions' in plotting_config:
        print(f"  region_exclusions: {len(plotting_config['region_exclusions'])} patterns")
        for pattern, vars_list in list(plotting_config['region_exclusions'].items())[:5]:
            print(f"    {pattern}: {len(vars_list)} variables")

    print("  ✓ Valid structure")

except Exception as e:
    print(f"✗ Error loading plotting.yaml: {e}")


In [ ]:
# Test configuration loading in PlotManager
print("\n=== Configuration Loading Test ===")
from darkbottomline.plotting import PlotManager

try:
    # Load with default config
    plot_manager_default = PlotManager()
    print("✓ PlotManager initialized with defaults")

    # Load with custom config
    if plotting_config_path.exists():
        plot_manager_custom = PlotManager(config=plotting_config)
        print("✓ PlotManager initialized with custom config")

        # Verify exclusions are loaded
        excluded = plot_manager_custom._get_excluded_variables_for_region("2b:CR_Top_mu")
        assert "z_mass" in excluded, "z_mass should be excluded from Top CR"
        assert "z_pt" in excluded, "z_pt should be excluded from Top CR"
        print("✓ Exclusions correctly applied from config")

except Exception as e:
    print(f"✗ Error testing configuration: {e}")
